# Morning Creek Temp Study

## Import Statements

In [1]:
import datetime as dt
import time

from os import makedirs

from flowmeterAnalysis import readFiles
from flowmeterAnalysis import flowMonitor
from flowmeterAnalysis import plotting

## File Locations

In [2]:
homeDir = 'P:\\PW-WATER SERVICES\\TECHNICAL SERVICES\\Anna'
flowDir = homeDir + '\\2019\\Morning Creek Temp Study\\Data'
upstreamFile = homeDir + '\\MCTemptStudy_fmus.csv'
fmdataFile = homeDir + '\\FMdata.csv'
rainDir = homeDir + '\\2019\\Morning Creek Temp Study\\Rain'
hourlyFile = rainDir + '\\MCRG01_hourly_02012019-04302019.csv'
dailyFile = rainDir + '\\MCRG01_daily_02012019-04302019.csv'

saveDir = homeDir + '\\2019\\Morning Creek Temp Study\\'

## Settings and Variable Inputs

In [3]:
# EMPTY VARIABLES:
dfFlows = {}
basinDryWeather = {}
gageStorms = {}
stormsDict = {}
systemII = {}

# SETTINGS AND VARIABLES
analysisDates = (dt.datetime(2019,2,16),dt.datetime(2019,4,16))
rainThresh = 0.1

## CODE

### read in rain files and flow monitor details as pandas dataframes

* dfDaily is the daily rain data with the datetime as an index
* dfHourly is the hourly rain data with the datetime as an index
* dfDetails contains information about the rain gauge, diameter, basin footprint, basin area, etc. for each flow monitor
* dfUpstream contains the immediate upstream flow monitors for each flow monitor

In [4]:
dfDaily = readFiles.readRaincsv(
    filename = dailyFile)
dfHourly = readFiles.readRaincsv(
    filename = hourlyFile)
dfDetails = readFiles.readFMdetails(
    filename = fmdataFile)
dfUpstream = readFiles.readUpstreamFile(
    filename = upstreamFile)

### find all the flow csvs

In [5]:
flow_folders,flow_txt,flow_csv = readFiles.findTextFiles(readDir=flowDir)

### construct basinDryWeatherDict

This is a python dictionary with the following keys and values:
* Weekday: a dictionary containing the following keys: 
    * d/D: a numpy array containing the depth over diameter values
    * Gross Q: a pandas series with datetimes as indices containing the gross flow rates
    * DataFrame: a pandas dataframe with time as the index, dates for the columns, and values corresponding to measured flow rates
* Weekend: a dictionary containing the following keys: 
    * d/D: a numpy array containing the depth over diameter values
    * Gross Q: a pandas series with datetimes as indices containing the gross flow rates
    * DataFrame: a pandas dataframe with time as the index, dates for the columns, and values corresponding to measured flow rates
* Overall: a dictionary containing the following keys: 
    * BI: a float of the base infiltration, defined as the groundwater infiltration as a percentage of the overall mean flow rate
    * d/D: a numpy array containing the depth over diameter values

In [6]:
tic = time.process_time()
for fmData in flow_csv:
    if fmData.startswith('TFM'):
        fmname = fmData.split('_')[0] + '-' + fmData.split('_')[1]
    else:
        fmname = fmData.split('_')[0]
    flowFile = flowDir + "\\" + fmData
    dfFlow = readFiles.readFlowviewcsv(filename = flowFile)
    dfFlows[fmname] = dfFlow.loc[
         analysisDates[0]:analysisDates[1] + dt.timedelta(hours=23,minutes=45)
         , :]
    basinDryWeather[fmname] = flowMonitor.dryWeather(
        dfFlow = dfFlows[fmname], 
        fmname = fmname, 
        dfDailyRain = dfDaily,
        rainColName = 'Rain (in)',
        dfmDetails = dfDetails, 
        analysisDates = analysisDates,
        rainThresh = rainThresh)
toc = time.process_time()
print('Processing time = ' + str(toc - tic))

Processing time = 3.4320220000000004


### Create stacked area graphs of the upstream flows and downstream monitor
The downstream monitor should always be higher than the sum of the upstream flows

In [7]:
tic = time.process_time()
folders, txt, csv = readFiles.findTextFiles(saveDir)
if 'Upstream Plots' not in folders:
    makedirs(saveDir + '\\Upstream Plots')
for fmname in dfFlows:
    usfmList = readFiles.findUpstreamFMs(
        df = dfUpstream, 
        fmname = fmname)
    if not usfmList:
        pass
    else:
        plotting.plotInstantUpstreamFlows(
            fmname = fmname, 
            dfFlows = dfFlows, 
            usfmList = usfmList,
            saveDir = saveDir + '\\Upstream Plots\\')
toc = time.process_time()
print('Processing time = ' + str(toc - tic))

Processing time = 5.616035999999999
